In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras

# 12.2 넘파이처럼 텐서플로 사용하기

## 12.2.1 텐서와 연산

In [14]:
tf.constant([[1.,2.,3.],[4.,5.,6.]])

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [15]:
tf.constant(42)

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [45]:
t = tf.constant([[1.,2.,3.],[4.,5.,6.]])
print(t.shape,"\n", t.dtype)

(2, 3) 
 <dtype: 'float32'>


In [17]:
t[:,1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [18]:
t[:,1,tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [19]:
t+10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [20]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [21]:
tf.matmul(t,tf.transpose(t))

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

## 12.2.2 텐서와 넘파이

 상호 변환, 상호 연산 적용이 자유롭다

In [22]:
a = np.array([2.,4.,5.])

In [23]:
t = tf.constant(a)
t

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [24]:
np.array(t)

array([2., 4., 5.])

In [25]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [26]:
np.square(t)

array([ 4., 16., 25.])

## 12.2.3 타입 변환

텐서플로는 자동으로 형변환을 해주지 않는다.  
텐서플로는 기본적을 32비트 정밀도를 사용한다.  
따라서 64비트와 연산 역시 자동으로 해주지 않는다.

In [27]:
try :tf.constant(2.) + tf.constant(40) 
except: (print("텐서플로는 자동형변환이 없음. 오류발생"))

텐서플로는 자동형변환이 없음. 오류발생


In [28]:
try :tf.constant(2.) + tf.constant(40, dtype = "float64")
except: print("비트 정밀도가 맞지 않음. 오류발생")

비트 정밀도가 맞지 않음. 오류발생


캐스팅하여 맞춰주면 진행.

In [29]:
t2 = tf.constant(40., dtype = "float64")
tf.constant(2.) + tf.cast(t2,tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

## 12.2.4 변수

In [30]:
v = tf.Variable([[1.,2.,3.],[4.,5.,6]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [31]:
v.assign(2*v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [32]:
v[0,1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [33]:
v[:,2].assign([0.,1.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>

# 12.3 사용자 정의 모델과 훈련 알고리즘

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [4]:
X_train.shape

(11610, 8)

In [5]:
input_shape = X_train.shape[1:]

In [6]:
input_shape

(8,)

In [7]:
model = keras.models.Sequential([
    keras.layers.Dense(300, activation = 'relu', kernel_initializer = 'he_normal',
                      input_shape = input_shape),
    keras.layers.Dense(100, activation = 'relu', kernel_initializer = 'he_normal'),
    keras.layers.Dense(1)
])

## 12.3.1 사용자 정의 손실 함수 

후버손실 구현.  
평균보다 하나의 샘플에 대한 계산을 정의하는 것이 유리.  
y_true, y_pred

In [38]:
def huber_loss(y_true, y_pred):
    error = y_true-y_pred
    is_small = tf.abs(error)<1
    mse = tf.square(error)/2
    mae = tf.abs(error) - 0.5
    return tf.where(is_small, mse, mae)

In [41]:
def create_huber(threshold):
    def huber_loss(y_true, y_pred):
        error = y_true-y_pred
        is_small = tf.abs(error)<threshold
        mse = tf.square(error)/2
        mae = tf.abs(error) - 0.5
        return tf.where(is_small, mse, mae)

후버손실(임계값에 따라 mse,mae 사용)

In [17]:
model.compile(loss = huber_loss,optimizer = 'nadam')

In [18]:
model.fit(X_train_scaled,y_train,epochs=10)

Epoch 1/10
363/363 [==============================] - 1s 3ms/step - loss: 0.3302
Epoch 2/10
363/363 [==============================] - 2s 5ms/step - loss: 0.2330
Epoch 3/10
363/363 [==============================] - 3s 8ms/step - loss: 0.2095
Epoch 4/10
363/363 [==============================] - 4s 11ms/step - loss: 0.1980
Epoch 5/10
363/363 [==============================] - 5s 13ms/step - loss: 0.2045
Epoch 6/10
363/363 [==============================] - 6s 16ms/step - loss: 0.1933
Epoch 7/10
363/363 [==============================] - 7s 19ms/step - loss: 0.1875
Epoch 8/10
363/363 [==============================] - 8s 21ms/step - loss: 0.1879
Epoch 9/10
363/363 [==============================] - 9s 24ms/step - loss: 0.1777
Epoch 10/10
363/363 [==============================] - 10s 27ms/step - loss: 0.1743


## 12.3.2 사용자 정의 요소를 가진 모델 저장하고 로드하기

In [19]:
model.save("mymodel.h5")

In [20]:
model = keras.models.load_model("mymodel.h5",
                               custom_objects={"huber_loss":huber_loss})

#### 파라미터를 저장해야 할때, 원래는 저장안됨

알맞는 클래스를 상속받아서 get config를 구현

In [26]:
class HuberLoss(keras.losses.Loss):
    def __init__(self,threshold= 1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self,y_ture, y_pred):
        error = y_true - y_pred
        is_small_eror = tf.abs(error) < self.threshold
        mse = tf.square(error) /2
        mae = self.threshold* tf.abs(error) -self.threshold**2 /2
        return tf.where(is_small_eror,mse,mae)
    ##중요
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold":self.threshold}
        

## 12.3.3 활성화 함수, 초기화, 규제, 제한을 커스터마이징 하기

softplus

In [27]:
def my_softplus(z):
    return tf.math.log(tf.exp(z + 1.0))

글로럿 = xavier

In [28]:
def my_glorot_initializer(shape, dtype = tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape,stddev=stddev,dtype = dtype)

l1

In [29]:
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))


positive clipping

In [30]:
def my_positive_weights(weights):
    return tf.where(weights<0, tf.zeros_like(weights), weights)

In [31]:
layer = keras.layers.Dense(30,
                          activation=  my_softplus,
                          kernel_initializer = my_glorot_initializer,
                          kernel_regularizer = my_l1_regularizer,
                          kernel_constraint = my_positive_weights)

하이퍼파라미터를 저장해야 할때,   

역시 알맞는 클래스를 상속받고, 

get config 구현

In [37]:
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self,factor):
        self.factor = factor
    def __call__(self,weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))

    def get_config(self):
        return {'factor' : self.factor}

## 12.3.4 사용자 정의 지표

지표와 손실함수는 사실 다르지 않음.  
지표는 이해할수 있어야하고  
손실은 이해하기 힘들어도 미분 가능하면 될 뿐. 

스트리밍 지표 만들기  
지표는 기본적을 매번 배치에서 구하고 평균을 냄  
하지만 정밀도 같은 지표는 매번 배치에서 새로 구해서 평균을 내면 누적값과 다르게 나옴. 

후버손실을 지표로 상속받아 구현할것인데,  
사실 후버손실은 정밀도와 다르게 배치마다 구해서 평균을 내도 결과가 같음.

In [42]:
class HuberMetric(keras.metrics.Metric):
    def __init__(self,threshold=1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold= threshold
        self.huber_fn =create_huber(threshold)
        self.total = self.add_weight("total", initializer = "zeros")
        self.count = self.add_weight("count", initializer = "zeros")
    def update_state(self,y_true,y_pred,sample_weight = None):
        metric = self.huber_fn(y_true,y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.total.assign_add(tf.cast(tf.size(y_true),tf.float32))
    def result(self):
        return self.total / self.count
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config,"threshold":self.threshold}
        

## 12.3.5 사용자 정의 층

In [239]:
from sklearn.model_selection import train_test_split

In [240]:
(X_train,y_train),(X_test,y_test) = keras.datasets.mnist.load_data()

In [241]:
X_train,X_valid,y_train,y_valid = train_test_split(X_train,y_train, test_size=  0.2)

In [242]:
X_train.shape

(48000, 28, 28)

In [243]:
y_train.shape

(48000,)

#### 가중치가 없는 층 --> lambda 로 처리  
ex) relu, flatten

In [143]:
exp_layer =  keras.layers.Lambda(lambda x : tf.exp(x))

#### 상태가 있는 층 --> keras.layers.Layer 상속

In [144]:
class MyDense(keras.layers.Layer):
    def __init__(self,unit,activation=None,**kwargs):
        super().__init__(**kwargs)
        self.unit = unit
        self.activation = keras.activations.get(activation)
    
    #build : add_weight 메서드를 호출하여 층의 변수를 만들어줌
    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name = "kernel", 
            shape = [batch_input_shape[-1], self.unit],
            initializer = keras.initializers.glorot_normal
        )
        self.bias = self.add_weight(
            name = "bias",
            shape = [self.unit],
            initializer = keras.initializers.zeros
        )
        #마지막의 부모의 build 메서드를 호출하여 층을 생성함을 알려줘야함 .
        super().build(batch_input_shape)
        
        
    #call 필요한 연산을 수행. 기본 dense 는 activation(입력 * Weights + bias)
    def call(self, X):
        return self.activation(tf.matmul(X,self.kernel) + self.bias)
    def compute_output_shape(self,batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1]+ [self.unit])
    
    def get_config(self):
        base_config  = super().get_config()
        return {**base_config, "unut": self.unit, 
                "activation": keras.activations.serialize(self.activation)}
        

In [145]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    MyDense(100,activation = 'relu'),
    MyDense(30,activation = 'relu'),
    MyDense(10,activation = 'softmax')
])

In [146]:
model.compile(loss= keras.losses.sparse_categorical_crossentropy,
             optimizer = keras.optimizers.Adam())

In [147]:
model.fit(X_train,y_train,epochs=5,
         validation_data  = (X_valid, y_valid))

Epoch 1/5
1500/1500 [==============================] - 1s 495us/step - loss: 4.8332 - val_loss: 1.3948
Epoch 2/5
1500/1500 [==============================] - 1s 446us/step - loss: 1.2631 - val_loss: 0.9182
Epoch 3/5
1500/1500 [==============================] - 1s 443us/step - loss: 0.8027 - val_loss: 0.5823
Epoch 4/5
1500/1500 [==============================] - 1s 438us/step - loss: 0.4873 - val_loss: 0.3363
Epoch 5/5
1500/1500 [==============================] - 1s 447us/step - loss: 0.2666 - val_loss: 0.2373


#### 입력 2개 출력 3개

In [148]:
class MymultiLayer(keras.layers.Layer):
    def call(self,inputs):
        X1,X2  = inputs
        
        return [X1+X2, X1*X2, X1/X2]
    
    def compute_output_shape(self, batch_input_shape):
        shape1, shape2 = batch_output_shape
        
        #브로드캐스팅 결과라서.
        return [shape1,shape1,shape1] 
        
        

#### 훈련과 테스트에서 다르게 동작하는 층

In [149]:
class MyGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev ,**kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev
        
    def call(self,X,training= None):
        if training:
            noise = tf.random.noraml(tf.shape(X), stddev = self.stddev)
            return X + noise
        else:
            return X
    def compute_output_shape(self,batch_input_shape):
        return batch_input_shape
        
        

## 12.3.6 사용자 정의 모델

잔차블럭(여러 레이어)을 사용하는 model 만들기

잔차블럭

In [272]:
class ResidualBlock(keras.layers.Layer):
    def __init__(self,n_layers,n_neurons,**kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons,activation = 'elu',
                                         kernel_initializer = "he_normal") 
                       for layer in range(n_layers)]
    def call(self,inputs):
        Z = inputs
        for layer in self.hidden:
            Z=  layer(Z)
        
        # 다 통과하고 인풋 더해줌.
        return inputs + Z

잔차블럭을 사용하는 모델

In [330]:
class ResidualRegressor(keras.Model):
    def __init__(self,output_dim,input_shape,**kwargs):
        super().__init__(**kwargs)
        self.flatten = keras.layers.Flatten(input_shape = input_shape)
        self.hidden1 = keras.layers.Dense(300,activation = 'elu',
                                         kernel_initializer=  "he_normal",
                                         )
        #인풋과 아웃풋 크기가 같아야 잔차 전달가능.
        
        self.block1 = ResidualBlock(2,300)
        self.block2 = ResidualBlock(2,300)
        self.out = keras.layers.Dense(output_dim)
        
        
    def call(self,X):
        Z = self.flatten(X)
        Z = self.hidden1(Z)
        for _ in range(4):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)

In [331]:
# input = keras.layers.Flatten()
res_model = ResidualRegressor(output_dim = 1,input_shape = [28,28])

In [333]:
res_model.compile(loss = 'mse',
                 optimizer = keras.optimizers.Adam())

In [334]:
res_model.fit(X_train,y_train,epochs=10,
             validation_data = (X_valid,y_valid))

Epoch 1/10
1500/1500 [==============================] - 3s 2ms/step - loss: 232135.3450 - val_loss: 154.1324
Epoch 2/10
1500/1500 [==============================] - 3s 2ms/step - loss: 178.4257 - val_loss: 46.1005
Epoch 3/10
1500/1500 [==============================] - 3s 2ms/step - loss: 72.3631 - val_loss: 22.9760
Epoch 4/10
1500/1500 [==============================] - 3s 2ms/step - loss: 53.6735 - val_loss: 12.6163
Epoch 5/10
1500/1500 [==============================] - 3s 2ms/step - loss: 30.3852 - val_loss: 9.1625
Epoch 6/10
1500/1500 [==============================] - 3s 2ms/step - loss: 19.6029 - val_loss: 9.0078
Epoch 7/10
1500/1500 [==============================] - 3s 2ms/step - loss: 11.9829 - val_loss: 5.4743
Epoch 8/10
1500/1500 [==============================] - 3s 2ms/step - loss: 10.4659 - val_loss: 4.9508
Epoch 9/10
1500/1500 [==============================] - 3s 2ms/step - loss: 5.6386 - val_loss: 10.5822
Epoch 10/10
1500/1500 [==============================] - 3s 2ms

In [335]:
res_model.evaluate(X_test,y_test)

313/313 [==============================] - 0s 618us/step - loss: 2.4566


2.4565823078155518

In [348]:
np.round(res_model.predict(X_test[:1]),0)

array([[7.]], dtype=float32)

In [349]:
y_test[:1]

array([7], dtype=uint8)

## 12.3.7 모델 구성 요소에 기반한 손실과 지표

#### reconstruction error 를 손실로 정의  
레이블과 예측으로 구하는게 아님 ==> 중간에 구해야함 ==> 서브클래싱으로 직접 계산

In [118]:
class ReconstructingRegressor(keras.Model):
    
    def __init__(self,output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(30,activation = 'selu', 
                                         kernel_initializer = 'lecun_normal') for layer in range(5)]
        self.out = keras.layers.Dense(output_dim)
        self.reconstruct = keras.layers.Dense(8)
        self.reconstruction_mean = keras.metrics.Mean(name = "reconstruction_error")
        
    
#     def build(self,batch_input_shape):

#         n_inputs = batch_input_shape[-1]
#         #결과가 인풋 크기랑 같아야 재구성오차 계산 가능
#         self.reconstruct = keras.layers.Dense(n_inputs)
#         #build 는 항상 부모의 build 메서드 호출하고 끝남
#         super().build(batch_input_shape)
        
    
    
    #tf 2.4 에 버그가 있음. build와 add_loss를 같이 못씀.
    
    def call(self,X,training = None):
        Z = X
        for layer in self.hidden:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        reconstruction_error = tf.reduce_mean(tf.square(reconstruction - X))
        self.add_loss(0.05 * reconstruction_error)
    
        result = self.reconstruction_mean(reconstruction_error)
        
        self.add_metric(result)
        return self.out(Z)
    

In [119]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [120]:
ReconstructingRegressor = ReconstructingRegressor(output_dim = 1)

In [121]:
ReconstructingRegressor.compile(loss = keras.losses.MSE,
                               optimizer = 'adam')

In [122]:
history = ReconstructingRegressor.fit(X_train_scaled,y_train,epochs=5,
                                     validation_data = (X_valid_scaled,y_valid))

Epoch 1/5
363/363 [==============================] - 0s 729us/step - loss: 1.4936 - reconstruction_error: 0.9945 - val_loss: 1.3449 - val_reconstruction_error: 14.2073
Epoch 2/5
363/363 [==============================] - 0s 553us/step - loss: 0.4683 - reconstruction_error: 0.4001 - val_loss: 0.9896 - val_reconstruction_error: 12.8080
Epoch 3/5
363/363 [==============================] - 0s 549us/step - loss: 0.4041 - reconstruction_error: 0.3089 - val_loss: 0.9919 - val_reconstruction_error: 12.2759
Epoch 4/5
363/363 [==============================] - 0s 575us/step - loss: 0.3718 - reconstruction_error: 0.2589 - val_loss: 0.8635 - val_reconstruction_error: 10.7394
Epoch 5/5
363/363 [==============================] - 0s 571us/step - loss: 0.3656 - reconstruction_error: 0.2214 - val_loss: 0.7992 - val_reconstruction_error: 9.2787


## 12.3.8 자동 미분을 사용하여 그레디언트 계산하기

In [123]:
def f(w1,w2):
    return 3*w1**2 + 2 *w1 *w2

In [125]:
w1,w2 = tf.Variable(5.), tf.Variable(3.)
with tf.GradientTape() as tape:
    z = f(w1,w2)
    
gradients = tape.gradient(z, [w1,w2])

In [126]:
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [127]:
try:gradients = tape.gradient(z, [w1,w2])
except : print("tape.gradient() 는 한번 호출되면 지워집니다")

tape.gradient() 는 한번 호출되면 지워집니다


한번이상 호출해야 될때는 persistent = true  
다쓰면 리소스 해제

In [130]:
with tf.GradientTape(persistent=True) as tape:
    z = f(w1,w2)

dz_dw1 = tape.gradient(z,w1)
dz_dw2 = tape.gradient(z,w2)

print(dz_dw1,dz_dw2)

del tape
    

tf.Tensor(36.0, shape=(), dtype=float32) tf.Tensor(10.0, shape=(), dtype=float32)


tf.Variable이 아닌 객체에 대한 편미분은 None 반환

In [131]:
c1,c2 = tf.constant(5.), tf.constant(3.)
with tf.GradientTape() as tape:
    z = f(c1,c2)
    
print(tape.gradient(z,[c1,c2]))
    

[None, None]


tf.Variable이 아니더라도 감시하여 강제로 기록할수 있음

In [132]:
c1,c2 = tf.constant(5.), tf.constant(3.)
with tf.GradientTape() as tape:
    
    ############
    tape.watch(c1)
    tape.watch(c2)
    ############
    z = f(c1,c2)
    
print(tape.gradient(z,[c1,c2]))
    

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>, <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]


일부가 역전파 되지 않게 하는법

In [133]:
def f(w1,w2):
    #정방향은 작동하지만 stop_gradient 안에는 역전파가 일어나지 않음.
    return 3*w1**2 + tf.stop_gradient(2*w1*w2)

In [134]:
with tf.GradientTape() as tape:
    ####### w2는 stop_gradient 안에만 있으니깐 역전파시 None 반환할듯.
    z = f(w1,w2)

In [135]:
gradient = tape.gradient(z,[w1,w2])

In [136]:
gradient

[<tf.Tensor: shape=(), dtype=float32, numpy=30.0>, None]

In [147]:
np.log(np.exp(40)+1)

40.0

## 12.3.9 사용자 정의 훈련 반복

.fit 보다 더 유연함을 원할때... ex) 두개의 옵티마이저.


더 어렵고, 버그가 잦으니 알아둘것.

In [148]:
l2_reg = keras.regularizers.l2(0.05)

In [187]:
model = keras.models.Sequential([
    keras.layers.Dense(30,activation = 'elu',kernel_initializer = 'he_normal',
                      kernel_regularizer = l2_reg),
    keras.layers.Dense(1, kernel_regularizer = l2_reg)
])

랜덤배치 추출함수 

In [188]:
def random_batch(X,y, batch_size= 32):
    idx = np.random.randint(len(X),size = batch_size)
    return X[idx],y[idx]

상태 바 출력

In [189]:
#깨알 팁. 캐리지 리턴 \r 
#같은 줄에 출력하면 밀어내고 새로출력. 상태 bar 만들때 유용
for i in range(10000):
    print(f"\r{i}",end = "")

9999

In [190]:
def print_status_bar(iteration, total, loss,metrics=None):
    metrics ='-'.join(["{}:{:.4f}".format(m.name,m.result())
                      for m in [loss]+(metrics or [] )])
    end = "" if iteration < total else "\n"
    print(f"\r{iteration}/{total} {metrics}",end = end)

하이퍼 파라미터 정의.

In [191]:
n_epochs = 5
batch_size=32
n_steps = len(X_train)//batch_size
optimizer = keras.optimizers.Adam(lr = 0.01)
loss_fn = keras.losses.mean_squared_error
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.MeanAbsoluteError()]

In [207]:
for epoch in range(1, n_epochs +1 ):
    print(f"에포크 {epoch}/{n_epochs}")
    
    for step in range(1,n_steps+1):
        batch_X, batch_y = random_batch(X_train,y_train,batch_size = batch_size)
        
        with tf.GradientTape() as tape:
            
            #히나의 배치에 대한 예측, 손실
            y_pred = model(batch_X)
            main_loss = tf.reduce_mean(loss_fn(batch_y,y_pred))
            loss = tf.add_n([main_loss] + model.losses)
        
        #하나의 배치에 대한 결과로 업데이트
        gradients = tape.gradient(loss,model.trainable_variables)
        optimizer.apply_gradients(zip(gradients,model.trainable_variables))
        
        for variable in model.variables:
            if variable.constraint is not None:
               variable.assign(variable.constraint(variable))
        
        #전체 배치 평균      
        mean_loss(loss)
        
        for metric in metrics:
            metric(batch_y,y_pred)
        print_status_bar(step *batch_size, len(y_train) , mean_loss, metrics)
        
    print_status_bar(len(y_train), len(y_train) , mean_loss, metrics)
    
    for metric in [mean_loss] + metrics:
        metric.reset_states()

        
    

에포크 1/5
11610/11610 mean:1.5085-mean_absolute_error:0.6599
에포크 2/5
11610/11610 mean:1.6632-mean_absolute_error:0.7270
에포크 3/5
11610/11610 mean:1.3941-mean_absolute_error:0.6620
에포크 4/5
11610/11610 mean:3.1613-mean_absolute_error:1.0377
에포크 5/5
11610/11610 mean:29.3634-mean_absolute_error:2.4586


# 12.4 텐서플로 함수와 그래프

텐서플로는 최적화를 진행하기 때문에  
파이썬 함수를 텐서플로 함수로 바꾸는 것이 좋음

In [208]:
def cube(X):
    return X **3

In [209]:
cube(2)

8

In [212]:
cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

텐서플로 함수로 바꾸기

In [214]:
tf_cube = tf.function(cube)
tf_cube

In [215]:
tf_cube(2)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [216]:
tf_cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

데코레이터를 사용하는게 더 대중적

In [217]:
@tf.function
def tf_cube(x):
    return x**3

In [219]:
#원본 파이썬 함수도 접근 가능
tf_cube.python_function(2)

8